# GP1 Trading Model

## Get the API Keys Ready

In [ ]:
from config_private import ALPACA_API_KEY, ALPACA_API_SECRET
API_BASE_URL = 'https://paper-api.alpaca.markets'

## Part 1: Imports and Class/Function definitions

In [ ]:
from config_tickers import GP1_TICKER
from config import INDICATORS
from config import CDL
from data_processor import DataProcessor
from plot import backtest_stats, backtest_plot, get_baseline, drop_dup_dates
import pickle
import numpy as np
import pandas as pd

## Environment

In [ ]:

import gymnasium as gym
import numpy as np
from numpy import random as rd


class StockTradingEnv(gym.Env):
    def __init__(
        self,
        config,
        gamma=0.985,
        turbulence_thresh=99,
        max_stock=None,
        min_stock_rate=0.2,
        initial_capital=1e5,
        reward_scaling=2**-11,
        sell_cost_pct=1e-2,
        initial_stocks=None,
    ):
        price_ary = config["price_array"]
        tech_ary = config["tech_array"]
        turbulence_ary = config["turbulence_array"]
        date_ary = config["date_array"]
        if_train = config["if_train"]
        self.price_ary = price_ary.astype(np.float32)
        self.tech_ary = tech_ary.astype(np.float32)
        self.turbulence_ary = turbulence_ary
        self.date_ary = date_ary

        self.tech_ary = self.tech_ary * 2**-7
        self.turbulence_bool = (turbulence_ary > turbulence_thresh).astype(np.float32)
        self.turbulence_ary = (
            self.sigmoid_sign(turbulence_ary, turbulence_thresh) * 2**-5
        ).astype(np.float32)

        stock_dim = self.price_ary.shape[1]
        self.gamma = gamma
        self.max_stock = max_stock
        self.min_stock_rate = min_stock_rate
        self.sell_cost_pct = sell_cost_pct
        self.reward_scaling = reward_scaling
        self.initial_capital = initial_capital
        self.initial_stocks = (
            np.zeros(stock_dim, dtype=np.float32)
            if initial_stocks is None
            else initial_stocks
        )
        
        # reset()
        self.minute = None
        self.num_trades = None
        self.cash = None
        self.stocks = None
        self.total_assets = None
        self.gamma_reward = None
        self.initial_total_assets = None

        # environment information
        self.env_name = "StockEnv"
        self.state_dim = 114 # Size of get_state() array 
        self.action_dim = stock_dim
        self.max_step = self.price_ary.shape[0] - 1
        self.if_train = if_train
        self.if_discrete = False
        self.episode_return = 0.0
        self.observation_space = gym.spaces.Box(
            low=-np.inf, high=np.inf, shape=(self.state_dim,), dtype=np.float32
        )
        self.action_space = gym.spaces.Box(
            low=-1, high=1, shape=(self.action_dim,), dtype=np.float32
        )
        
    def reset(self, seed=None, options=None):
        super().reset(seed=seed)
        self.minute = 0
        self.num_trades = 0
        price = self.price_ary[self.minute]
        
        if self.if_train:
            self.stocks = (
                self.initial_stocks + rd.randint(0, 17, size=self.initial_stocks.shape)
            ).astype(np.float32)
            self.stocks_cool_down = np.zeros_like(self.stocks)
            self.cash = (
                self.initial_capital * rd.uniform(0.95, 1.05)
                - (self.stocks * price).sum()
            )
        else:
            self.stocks = self.initial_stocks.astype(np.float32)
            self.stocks_cool_down = np.zeros_like(self.stocks)
            self.cash = self.initial_capital
        
        self.total_assets = self.cash + (self.stocks * price).sum()
        self.initial_total_assets = self.total_assets
        self.gamma_reward = 0.0
        observation = self.get_state(price)
        info = {}
        return observation, info

    def step(self, action):
        self.minute += 1
        price = self.price_ary[self.minute]
        self.max_stock = np.round(np.floor(50_500 / price)).astype(int)
        action = np.round((action * self.max_stock)).astype(int)
        min_action = np.round((self.max_stock * self.min_stock_rate)).astype(int)
        self.stocks_cool_down += 1
        penalty = 0
        
        if self.turbulence_bool[self.minute] == 0:
            
            # Sell Logic
            for index in np.where((action < -min_action) & (self.stocks_cool_down > 4))[0]:
                if price[index] > 0:
                    sell_num_shares = min(self.stocks[index], -action[index])
                    sell_value = price[index] * sell_num_shares
                    self.stocks[index] -= sell_num_shares
                    self.cash += sell_value
                    penalty += sell_value * self.sell_cost_pct
                    self.stocks_cool_down[index] = 0
                    self.num_trades += 1

            # Buy Logic
            for index in np.where((action > min_action) & (self.stocks_cool_down > 4))[0]:
                if price[index] > 0:
                    buy_num_shares = min(self.cash // price[index], action[index])
                    buy_value = price[index] * buy_num_shares
                    self.stocks[index] += buy_num_shares
                    self.cash -= buy_value
                    self.stocks_cool_down[index] = 0
                    self.num_trades += 1

        # turbulence logic
        else:
            self.cash += (self.stocks * price).sum()
            self.num_trades += np.count_nonzero(self.stocks)
            self.stocks[:] = 0
            self.stocks_cool_down[:] = 0
        
        # Reward Calculations
        observation = self.get_state(price)
        total_assets = self.cash + (self.stocks * price).sum()
        reward = ((total_assets - penalty) - self.total_assets) * self.reward_scaling
        self.total_assets = total_assets
        self.gamma_reward = self.gamma_reward * self.gamma + reward
        terminated = self.minute == self.max_step
        truncated = False
        info = {}
        if terminated:
            reward = self.gamma_reward
            self.episode_return = total_assets / self.initial_total_assets

        return observation, reward, terminated, truncated, info

    def get_state(self, price):
        cash = np.array(self.cash * (2**-12), dtype=np.float32)
        scale = np.array(2**-6, dtype=np.float32)
        observation = np.hstack(
            (
                cash,
                self.turbulence_ary[self.minute],
                self.turbulence_bool[self.minute],
                price * scale,
                self.stocks * scale,
                self.stocks_cool_down,
                self.tech_ary[self.minute],
            )
        )
        # print(len(observation))
        return observation

    @staticmethod
    def sigmoid_sign(ary, thresh):
        def sigmoid(x):
            return 1 / (1 + np.exp(-x * np.e)) - 0.5

        return sigmoid(ary / thresh) * thresh

## PPO

In [ ]:
import os
import time
import gymnasium as gym
import numpy as np
import numpy.random as rd
import torch
import torch.nn as nn
from copy import deepcopy
from torch import Tensor
from torch.distributions.normal import Normal
from torch.nn.utils import clip_grad_norm_


class ActorPPO(nn.Module):
    def __init__(self, dims: [int], state_dim: int, action_dim: int):
        super().__init__()
        self.net = build_mlp(dims=[state_dim, *dims, action_dim])
        layer_init_with_orthogonal(self.net[-1], std=0.1)
        self.action_std_log = nn.Parameter(torch.zeros((1, action_dim)), requires_grad=True)  # trainable parameter

    def forward(self, state: Tensor) -> Tensor:
        return self.net(state).tanh()  # action.tanh()

    def get_action(self, state: Tensor) -> (Tensor, Tensor):  # for exploration
        action_avg = self.net(state)
        action_std = self.action_std_log.exp()

        dist = Normal(action_avg, action_std)
        action = dist.sample()
        logprob = dist.log_prob(action).sum(1)
        return action, logprob

    def get_logprob_entropy(self, state: Tensor, action: Tensor) -> (Tensor, Tensor):
        action_avg = self.net(state)
        action_std = self.action_std_log.exp()

        dist = Normal(action_avg, action_std)
        logprob = dist.log_prob(action).sum(1)
        entropy = dist.entropy().sum(1)
        return logprob, entropy

    @staticmethod
    def convert_action_for_env(action: Tensor) -> Tensor:
        return action.tanh()


class CriticPPO(nn.Module):
    def __init__(self, dims: [int], state_dim: int, _action_dim: int):
        super().__init__()
        self.net = build_mlp(dims=[state_dim, *dims, 1])
        layer_init_with_orthogonal(self.net[-1], std=0.5)

        self.state_avg = nn.Parameter(torch.zeros((state_dim,)), requires_grad=False)
        self.state_std = nn.Parameter(torch.ones((state_dim,)), requires_grad=False)
        self.value_avg = nn.Parameter(torch.zeros((1,)), requires_grad=False)
        self.value_std = nn.Parameter(torch.ones((1,)), requires_grad=False)

    def state_norm(self, state: Tensor) -> Tensor:
        return (state - self.state_avg) / self.state_std

    def value_re_norm(self, value: Tensor) -> Tensor:
        return value * self.value_std + self.value_avg

    def forward(self, state: Tensor) -> Tensor:
        state = self.state_norm(state)
        value = self.net(state)
        value = self.value_re_norm(value)
        return value  # state value used to calculate advantage value


def build_mlp(dims: [int]) -> nn.Sequential:  # MLP (MultiLayer Perceptron)
    net_list = []
    for i in range(len(dims) - 1):
        net_list.extend([nn.Linear(dims[i], dims[i + 1]), nn.ReLU()])
    del net_list[-1]  # remove the activation of output layer
    return nn.Sequential(*net_list)

def layer_init_with_orthogonal(layer, std=1.0, bias_const=1e-6):
    torch.nn.init.orthogonal_(layer.weight, std)
    torch.nn.init.constant_(layer.bias, bias_const)

class Config:
    def __init__(self, agent_class=None, env_class=None, env_args=None):
        self.env_class = env_class  # env = env_class(**env_args)
        self.env_args = env_args  # env = env_class(**env_args)

        if env_args is None:  # dummy env_args
            env_args = {'env_name': None, 'state_dim': None, 'action_dim': None, 'if_discrete': None}
        self.env_name = env_args['env_name']  # the name of environment. Be used to set 'cwd'.
        self.state_dim = env_args['state_dim']  # vector dimension (feature number) of state
        self.action_dim = env_args['action_dim']  # vector dimension (feature number) of action
        self.if_discrete = env_args['if_discrete']  # discrete or continuous action space

        self.agent_class = agent_class  # agent = agent_class(...)

        '''Arguments for reward shaping'''
        self.gamma = 0.99  # discount factor of future rewards
        self.reward_scale = 2 ** 0  # an approximate target reward usually be closed to 256

        '''Arguments for training'''
        self.gpu_id = int(0)  # `int` means the ID of single GPU, -1 means CPU
        self.net_dims = (128, 64)  # the middle layer dimension of MLP (MultiLayer Perceptron)
        self.learning_rate = 6e-5  # 2 ** -14 ~= 6e-5
        self.weight_decay = 1e-2  # Default value for weight_decay
        self.lambda_entropy = 1e-2  # Default value for lambda_entropy
        self.soft_update_tau = 5e-3  # 2 ** -8 ~= 5e-3
        self.batch_size = int(512)  # num of transitions sampled from replay buffer.
        self.horizon_len = int(2000)  # collect horizon_len step while exploring, then update network
        self.buffer_size = None  # ReplayBuffer size. Empty the ReplayBuffer for on-policy.
        self.repeat_times = 8.0  # repeatedly update network using ReplayBuffer to keep critic's loss small

        '''Arguments for evaluate'''
        self.cwd = None  # current working directory to save model. None means set automatically
        self.break_step = +np.inf  # break training if 'total_step > break_step'
        self.eval_times = int(3)  # number of times that get episodic cumulative return
        self.eval_per_step = int(2e4)  # evaluate the agent per training steps

    def init_before_training(self):
        if self.cwd is None:  # set cwd (current working directory) for saving model
            self.cwd = f'./{self.env_name}_{self.agent_class.__name__[5:]}'
        os.makedirs(self.cwd, exist_ok=True)


def get_gym_env_args(env, if_print: bool) -> dict:
    if {'unwrapped', 'observation_space', 'action_space', 'spec'}.issubset(dir(env)):  # isinstance(env, gym.Env):
        env_name = env.unwrapped.spec.id
        state_shape = env.observation_space.shape
        state_dim = state_shape[0] if len(state_shape) == 1 else state_shape  # sometimes state_dim is a list

        if_discrete = isinstance(env.action_space, gym.spaces.Discrete)
        if if_discrete:  # make sure it is discrete action space
            action_dim = env.action_space.n
        elif isinstance(env.action_space, gym.spaces.Box):  # make sure it is continuous action space
            action_dim = env.action_space.shape[0]

    env_args = {'env_name': env_name, 'state_dim': state_dim, 'action_dim': action_dim, 'if_discrete': if_discrete}
    print(f"env_args = {repr(env_args)}") if if_print else None
    return env_args


def kwargs_filter(function, kwargs: dict) -> dict:
    import inspect
    sign = inspect.signature(function).parameters.values()
    sign = {val.name for val in sign}
    common_args = sign.intersection(kwargs.keys())
    return {key: kwargs[key] for key in common_args}  # filtered kwargs


def build_env(env_class=None, env_args=None):
    if env_class.__module__ == 'gym.envs.registration':  # special rule
        env = env_class(id=env_args['env_name'])
    else:
        env = env_class(**kwargs_filter(env_class.__init__, env_args.copy()))
    for attr_str in ('env_name', 'state_dim', 'action_dim', 'if_discrete'):
        setattr(env, attr_str, env_args[attr_str])
    return env


class AgentBase:
    def __init__(self, net_dims: [int], state_dim: int, action_dim: int, gpu_id: int = 0, args: Config = Config()):
        self.state_dim = state_dim
        self.action_dim = action_dim

        self.gamma = args.gamma
        self.batch_size = args.batch_size
        self.repeat_times = args.repeat_times
        self.reward_scale = args.reward_scale
        self.soft_update_tau = args.soft_update_tau

        self.states = None  # assert self.states == (1, state_dim)
        self.device = torch.device(f"cuda:{gpu_id}" if (torch.cuda.is_available() and (gpu_id >= 0)) else "cpu")

        act_class = getattr(self, "act_class", None)
        cri_class = getattr(self, "cri_class", None)
        self.act = self.act_target = act_class(net_dims, state_dim, action_dim).to(self.device)
        self.cri = self.cri_target = cri_class(net_dims, state_dim, action_dim).to(self.device) \
            if cri_class else self.act

        self.act_optimizer = torch.optim.RAdam(self.act.parameters(), lr=args.learning_rate, weight_decay=args.weight_decay)
        self.cri_optimizer = torch.optim.RAdam(self.cri.parameters(), lr=args.learning_rate, weight_decay=args.weight_decay) \
            if cri_class else self.act_optimizer

        self.criterion = torch.nn.SmoothL1Loss()

    @staticmethod
    def optimizer_update(optimizer, objective: Tensor):
        optimizer.zero_grad()
        objective.backward()
        clip_grad_norm_(parameters=optimizer.param_groups[0]["params"], max_norm=3.0)  # Add gradient clipping
        optimizer.step()

    @staticmethod
    def soft_update(target_net: torch.nn.Module, current_net: torch.nn.Module, tau: float):
        for tar, cur in zip(target_net.parameters(), current_net.parameters()):
            tar.data.copy_(cur.data * tau + tar.data * (1.0 - tau))


class AgentPPO(AgentBase):
    def __init__(self, net_dims: [int], state_dim: int, action_dim: int, gpu_id: int = 0, args: Config = Config()):
        self.if_off_policy = False
        self.act_class = getattr(self, "act_class", ActorPPO)
        self.cri_class = getattr(self, "cri_class", CriticPPO)
        AgentBase.__init__(self, net_dims, state_dim, action_dim, gpu_id, args)

        self.ratio_clip = getattr(args, "ratio_clip", 0.25)  # `ratio.clamp(1 - clip, 1 + clip)`
        self.lambda_gae_adv = getattr(args, "lambda_gae_adv", 0.95)  # could be 0.80~0.99
        self.lambda_entropy = getattr(args, "lambda_entropy", 1e-2)  # could be 0.00~0.10
        self.lambda_entropy = torch.tensor(self.lambda_entropy, dtype=torch.float32, device=self.device)

    def explore_env(self, env, horizon_len: int) -> [Tensor]:
        states = torch.zeros((horizon_len, self.state_dim), dtype=torch.float32).to(self.device)
        actions = torch.zeros((horizon_len, self.action_dim), dtype=torch.float32).to(self.device)
        logprobs = torch.zeros(horizon_len, dtype=torch.float32).to(self.device)
        rewards = torch.zeros(horizon_len, dtype=torch.float32).to(self.device)
        dones = torch.zeros(horizon_len, dtype=torch.bool).to(self.device)

        ary_state = self.states[0]

        get_action = self.act.get_action
        convert = self.act.convert_action_for_env
        for i in range(horizon_len):
            state = torch.as_tensor(ary_state, dtype=torch.float32, device=self.device)
            action, logprob = [t.squeeze(0) for t in get_action(state.unsqueeze(0))[:2]]

            ary_action = convert(action).detach().cpu().numpy()
            ary_state, reward, done, _, _ = env.step(ary_action)
            if done:
                ary_state, _ = env.reset()

            states[i] = state
            actions[i] = action
            logprobs[i] = logprob
            rewards[i] = reward
            dones[i] = done

        self.states[0] = ary_state
        rewards = (rewards * self.reward_scale).unsqueeze(1)
        undones = (1 - dones.type(torch.float32)).unsqueeze(1)
        return states, actions, logprobs, rewards, undones

    def update_net(self, buffer) -> [float]:
        with torch.no_grad():
            states, actions, logprobs, rewards, undones = buffer
            buffer_size = states.shape[0]

            '''get advantages reward_sums'''
            bs = 2 ** 10  # set a smaller 'batch_size' when out of GPU memory.
            values = [self.cri(states[i:i + bs]) for i in range(0, buffer_size, bs)]
            values = torch.cat(values, dim=0).squeeze(1)  # values.shape == (buffer_size, )

            advantages = self.get_advantages(rewards, undones, values)  # advantages.shape == (buffer_size, )
            reward_sums = advantages + values  # reward_sums.shape == (buffer_size, )
            del rewards, undones, values

            advantages = (advantages - advantages.mean()) / (advantages.std(dim=0) + 1e-5)
        assert logprobs.shape == advantages.shape == reward_sums.shape == (buffer_size,)

        '''update network'''
        obj_critics = 0.0
        obj_actors = 0.0

        update_times = int(buffer_size * self.repeat_times / self.batch_size)
        assert update_times >= 1
        for _ in range(update_times):
            indices = torch.randint(buffer_size, size=(self.batch_size,), requires_grad=False)
            state = states[indices]
            action = actions[indices]
            logprob = logprobs[indices]
            advantage = advantages[indices]
            reward_sum = reward_sums[indices]

            value = self.cri(state).squeeze(1)  # critic network predicts the reward_sum (Q value) of state
            obj_critic = self.criterion(value, reward_sum)
            self.optimizer_update(self.cri_optimizer, obj_critic)

            new_logprob, obj_entropy = self.act.get_logprob_entropy(state, action)
            ratio = (new_logprob - logprob.detach()).exp()
            surrogate1 = advantage * ratio
            surrogate2 = advantage * ratio.clamp(1 - self.ratio_clip, 1 + self.ratio_clip)
            obj_surrogate = torch.min(surrogate1, surrogate2).mean()

            obj_actor = obj_surrogate + obj_entropy.mean() * self.lambda_entropy
            self.optimizer_update(self.act_optimizer, -obj_actor)

            obj_critics += obj_critic.item()
            obj_actors += obj_actor.item()
        a_std_log = getattr(self.act, 'a_std_log', torch.zeros(1)).mean()
        return obj_critics / update_times, obj_actors / update_times, a_std_log.item()

    def get_advantages(self, rewards: Tensor, undones: Tensor, values: Tensor) -> Tensor:
        advantages = torch.empty_like(values)  # advantage value

        masks = undones * self.gamma
        horizon_len = rewards.shape[0]

        next_state = torch.tensor(self.states, dtype=torch.float32).to(self.device)
        next_value = self.cri(next_state).detach()[0, 0]

        advantage = 0  # last_gae_lambda
        for t in range(horizon_len - 1, -1, -1):
            delta = rewards[t] + masks[t] * next_value - values[t]
            advantages[t] = advantage = delta + masks[t] * self.lambda_gae_adv * advantage
            next_value = values[t]
        return advantages

    
def train_agent(args: Config):
    args.init_before_training()

    env = build_env(args.env_class, args.env_args)
    agent = args.agent_class(args.net_dims, args.state_dim, args.action_dim, gpu_id=args.gpu_id, args=args)
    agent.states, _ = env.reset()
    agent.states = agent.states[np.newaxis, :]

    evaluator = Evaluator(eval_env=build_env(args.env_class, args.env_args),
                          eval_per_step=args.eval_per_step,
                          eval_times=args.eval_times,
                          cwd=args.cwd)
    torch.set_grad_enabled(False)
    while True: # start training
        buffer_items = agent.explore_env(env, args.horizon_len)

        torch.set_grad_enabled(True)
        logging_tuple = agent.update_net(buffer_items)
        torch.set_grad_enabled(False)

        evaluator.evaluate_and_save(agent.act, args.horizon_len, logging_tuple)
        if (evaluator.total_step > args.break_step) or os.path.exists(f"{args.cwd}/stop"):
            torch.save(agent.act.state_dict(), args.cwd + '/actor.pth')
            break  # stop training when reach `break_step` or `mkdir cwd/stop`


def render_agent(env_class, env_args: dict, net_dims: [int], agent_class, actor_path: str, render_times: int = 8):
    env = build_env(env_class, env_args)

    state_dim = env_args['state_dim']
    action_dim = env_args['action_dim']
    agent = agent_class(net_dims, state_dim, action_dim, gpu_id=-1)
    actor = agent.act

    print(f"| render and load actor from: {actor_path}")
    actor.load_state_dict(torch.load(actor_path, map_location=lambda storage, loc: storage))
    for i in range(render_times):
        cumulative_reward, episode_step = get_rewards_and_steps(env, actor, if_render=True)
        print(f"|{i:4}  cumulative_reward {cumulative_reward:9.3f}  episode_step {episode_step:5.0f}")

        
class Evaluator:
    def __init__(self, eval_env, eval_per_step: int = 1e4, eval_times: int = 8, cwd: str = '.'):
        self.cwd = cwd
        self.env_eval = eval_env
        self.eval_step = 0
        self.total_step = 0
        self.start_time = time.time()
        self.eval_times = eval_times  # number of times that get episodic cumulative return
        self.eval_per_step = eval_per_step  # evaluate the agent per training steps

        self.recorder = []
        print(f"\n| step: Number of samples, or total training steps, or running times of `env.step()`."
              f"\n| time: Time spent from the start of training to this moment."
              f"\n| avgR: Average value of cumulative rewards, which is the sum of rewards in an episode."
              f"\n| stdR: Standard dev of cumulative rewards, which is the sum of rewards in an episode."
              f"\n| avgS: Average of steps in an episode."
              f"\n| objC: Objective of Critic network. Or call it loss function of critic network."
              f"\n| objA: Objective of Actor network. It is the average Q value of the critic network."
              f"\n| {'step':>8}  {'time':>8}  | {'avgR':>8}  {'stdR':>6}  {'avgS':>6}  | {'objC':>8}  {'objA':>8}")
            
    def evaluate_and_save(self, actor, horizon_len: int, logging_tuple: tuple):
        self.total_step += horizon_len
        if self.eval_step + self.eval_per_step > self.total_step:
            return
        self.eval_step = self.total_step

        rewards_steps_ary = [get_rewards_and_steps(self.env_eval, actor) for _ in range(self.eval_times)]
        rewards_steps_ary = np.array(rewards_steps_ary, dtype=np.float32)
        avg_r = rewards_steps_ary[:, 0].mean()  # average of cumulative rewards
        std_r = rewards_steps_ary[:, 0].std()  # std of cumulative rewards
        avg_s = rewards_steps_ary[:, 1].mean()  # average of steps in an episode

        used_time = time.time() - self.start_time
        self.recorder.append((self.total_step, used_time, avg_r))
        
        print(f"| {self.total_step:8.2e}  {used_time:8.0f}  "
              f"| {avg_r:8.2f}  {std_r:6.2f}  {avg_s:6.0f}  "
              f"| {logging_tuple[0]:8.2f}  {logging_tuple[1]:8.2f}")


def get_rewards_and_steps(env, actor, if_render: bool = False) -> (float, int):  # cumulative_rewards and episode_steps
    device = next(actor.parameters()).device  # net.parameters() is a Python generator.

    state, _ = env.reset()
    episode_steps = 0
    cumulative_returns = 0.0  # sum of rewards in an episode
    for episode_steps in range(100_000):
        tensor_state = torch.as_tensor(state, dtype=torch.float32, device=device).unsqueeze(0)
        tensor_action = actor(tensor_state)
        action = tensor_action.detach().cpu().numpy()[0]  # not need detach(), because using torch.no_grad() outside
        state, reward, done, _, _ = env.step(action)
        cumulative_returns += reward

        if if_render:
            env.render()
        if done:
            break
    return cumulative_returns, episode_steps + 1

## DRL Agent Class

In [ ]:
import torch

MODELS = {"ppo": AgentPPO}
OFF_POLICY_MODELS = ["ddpg", "td3", "sac"]
ON_POLICY_MODELS = ["ppo"]



class DRLAgent:
    """Implementations of DRL algorithms
    Attributes
    ----------
        env: gym environment class
            user-defined class
    Methods
    -------
        get_model()
            setup DRL algorithms
        train_model()
            train DRL algorithms in a train dataset
            and output the trained model
        DRL_prediction()
            make a prediction in a test dataset and get results
    """

    def __init__(self, env, price_array, tech_array, turbulence_array, date_array):
            self.env = env
            self.price_array = price_array
            self.tech_array = tech_array
            self.turbulence_array = turbulence_array
            self.date_array = date_array

    def get_model(self, model_name, model_kwargs, cwd, model_resume_training=True):
        env_config = {
            "price_array": self.price_array,
            "tech_array": self.tech_array,
            "turbulence_array": self.turbulence_array,
            "date_array": self.date_array,
            "if_train": True,
        }
        environment = self.env(config=env_config)
        env_args = {'config': env_config,
              'env_name': environment.env_name,
              'state_dim': environment.state_dim,
              'action_dim': environment.action_dim,
              'if_discrete': False}
        agent = MODELS[model_name]
        if model_name not in MODELS:
            raise NotImplementedError("NotImplementedError")
        model = Config(agent_class=agent, env_class=self.env, env_args=env_args)
        model.cwd = cwd
        model.if_off_policy = model_name in OFF_POLICY_MODELS
        if model_kwargs is not None:
            try:
                model.learning_rate = model_kwargs["learning_rate"]
                model.batch_size = model_kwargs["batch_size"]
                model.gamma = model_kwargs["gamma"]
                model.seed = model_kwargs["seed"]
                model.net_dims = model_kwargs["net_dimension"]
                model.target_step = model_kwargs["target_step"]
                model.eval_gap = model_kwargs["eval_gap"]
                model.eval_times = model_kwargs["eval_times"]
                model.weight_decay = model_kwargs["weight_decay"]
                model.lambda_entropy = model_kwargs["lambda_entropy"]
            except BaseException:
                raise ValueError(
                    "Fail to read arguments, please check 'model_kwargs' input."
                )
        # Initialize the agent's act attribute
        agent_instance = agent(model_kwargs["net_dimension"], environment.state_dim, environment.action_dim)
        model.act = agent_instance.act
        # Resume trainining or start from scratch logic
        model_path = model.cwd + '/actor.pth'
        if model_resume_training and os.path.exists(model_path):
            print(f"Loading existing model from: {model_path}")
            model.act.load_state_dict(torch.load(model_path, map_location=lambda storage, loc: storage))
        elif not model_resume_training and os.path.exists(model_path):
            print(f"Deleting existing model and starting training from scratch.")
            os.remove(model_path)
        else:
            print("No existing model found. Starting training from scratch.")
        
        return model

    def train_model(self, model, cwd, total_timesteps=5000):
        model.cwd = cwd
        model.break_step = total_timesteps
        train_agent(model)

    @staticmethod
    def DRL_prediction(model_name, cwd, net_dimension, environment):
        if model_name not in MODELS:
            raise NotImplementedError("NotImplementedError")
        agent_class = MODELS[model_name]
        environment.env_num = 1
        agent = agent_class(net_dimension, environment.state_dim, environment.action_dim)
        actor = agent.act
        # load agent
        try:  
            cwd = cwd + '/actor.pth'
            print(f"| load actor from: {cwd}")
            actor.load_state_dict(torch.load(cwd, map_location=lambda storage, loc: storage))
            act = actor
            device = agent.device
        except BaseException:
            raise ValueError("Fail to load agent!")

        # test on the testing env
        _torch = torch
        state, _ = environment.reset()
        episode_returns = []  # the cumulative_return / initial_account
        episode_total_assets = [environment.initial_total_assets]
        episode_dates = []
        with _torch.no_grad():
            for i in range(environment.max_step):
                s_tensor = _torch.as_tensor((state,), device=device)
                a_tensor = act(s_tensor)  # action_tanh = act.forward()
                action = (
                    a_tensor.detach().cpu().numpy()[0]
                )  # not need detach(), because with torch.no_grad() outside
                state, reward, done, _, _ = environment.step(action)
                # episode_dates.append(environment.get_date())
                episode_dates.append(environment.date_ary[environment.minute])
                total_assets = (
                    environment.cash
                    + (
                        environment.price_ary[environment.minute] * environment.stocks
                    ).sum()
                )
                episode_total_assets.append(total_assets)
                episode_return = total_assets / environment.initial_total_assets
                episode_returns.append(episode_return)
                if done:
                    break
        num_trades = environment.num_trades
        print("Test Finished!")
        # return episode total_assets on testing data
        print("episode_return", episode_return)
        return episode_dates, episode_total_assets, num_trades

## Train Function

In [ ]:
def train(
    start_date,
    end_date,
    ticker_list,
    data_source,
    time_interval,
    technical_indicator_list,
    drl_lib,
    env,
    model_name,
    if_vix=True,
    if_cdl=True,
    **kwargs,
):

    # Create 'train_data' folder in the current working directory if it doesn't exist
    folder_path = os.path.join(os.getcwd(), "train_data")
    os.makedirs(folder_path, exist_ok=True)

    # Set the file paths within the 'train_data' folder
    data_file = os.path.join(folder_path, f"data_{start_date}_{end_date}.pkl")
    arrays_file = os.path.join(folder_path, f"arrays_{start_date}_{end_date}.pkl")

    if os.path.exists(data_file) and os.path.exists(arrays_file):
        # Load the saved data
        with open(data_file, "rb") as f:
            data = pickle.load(f)
        with open(arrays_file, "rb") as f:
            price_array, tech_array, turbulence_array, date_array = pickle.load(f)
    
    else:
        # download data
        dp = DataProcessor(data_source, **kwargs)
        data = dp.download_data(ticker_list, start_date, end_date, time_interval)
        data = dp.clean_data(data)
        data = dp.add_technical_indicator(data, technical_indicator_list)
        if if_cdl:
            data = dp.add_cdl(data)
        if if_vix:
            data = dp.add_vix(data)
        else:
            data = dp.add_turbulence(data)
        price_array, tech_array, turbulence_array, date_array = dp.df_to_array(data, if_vix, if_cdl)
        
        # Save the data and arrays
        with open(data_file, "wb") as f:
            pickle.dump(data, f)
        with open(arrays_file, "wb") as f:
            pickle.dump((price_array, tech_array, turbulence_array, date_array), f)
            
    env_config = {
        "price_array": price_array,
        "tech_array": tech_array,
        "turbulence_array": turbulence_array,
        "date_array": date_array,
        "if_train": True,
    }
    env_instance = env(config=env_config)

    # read parameters
    cwd = kwargs.get("cwd", "./" + str(model_name))

    if drl_lib == "elegantrl":
        DRLAgent_erl = DRLAgent
        break_step = kwargs.get("break_step", 1e6)
        erl_params = kwargs.get("erl_params")
        agent = DRLAgent_erl(
            env=env,
            price_array=price_array,
            tech_array=tech_array,
            turbulence_array=turbulence_array,
            date_array=date_array,
        )
        model = agent.get_model(model_name, model_kwargs=erl_params, cwd=cwd)
        trained_model = agent.train_model(
            model=model, cwd=cwd, total_timesteps=break_step
        )    

## Test Function

In [ ]:
def test(
    start_date,
    end_date,
    ticker_list,
    data_source,
    time_interval,
    technical_indicator_list,
    drl_lib,
    env,
    model_name,
    if_vix=True,
    if_cdl=True,
    **kwargs,
):

    # Create 'test_data' folder in the current working directory if it doesn't exist
    folder_path = os.path.join(os.getcwd(), "test_data")
    os.makedirs(folder_path, exist_ok=True)

    # Set the file paths within the 'test_data' folder
    data_file = os.path.join(folder_path, f"data_{start_date}_{end_date}.pkl")
    arrays_file = os.path.join(folder_path, f"arrays_{start_date}_{end_date}.pkl")

    if os.path.exists(data_file) and os.path.exists(arrays_file):
        # Load the saved data
        with open(data_file, "rb") as f:
            data = pickle.load(f)
        with open(arrays_file, "rb") as f:
            price_array, tech_array, turbulence_array, date_array = pickle.load(f)
    
    else:
        # download data
        dp = DataProcessor(data_source, **kwargs)
        data = dp.download_data(ticker_list, start_date, end_date, time_interval)
        data = dp.clean_data(data)
        data = dp.add_technical_indicator(data, technical_indicator_list)
        if if_cdl:
            data = dp.add_cdl(data)
        if if_vix:
            data = dp.add_vix(data)
        else:
            data = dp.add_turbulence(data)
        price_array, tech_array, turbulence_array, date_array = dp.df_to_array(data, if_vix, if_cdl)
        
        # Save the data and arrays
        with open(data_file, "wb") as f:
            pickle.dump(data, f)
        with open(arrays_file, "wb") as f:
            pickle.dump((price_array, tech_array, turbulence_array, date_array), f)
            
    env_config = {
        "price_array": price_array,
        "tech_array": tech_array,
        "turbulence_array": turbulence_array,
        "date_array": date_array,
        "if_train": False,
    }
    env_instance = env(config=env_config)

    # load elegantrl needs state dim, action dim and net dim
    net_dimension = kwargs.get("net_dimension", 2**7)
    cwd = kwargs.get("cwd", "./" + str(model_name))
    print("price_array: ", len(price_array))

    if drl_lib == "elegantrl":
        DRLAgent_erl = DRLAgent
        episode_dates, episode_total_assets, num_trades = DRLAgent_erl.DRL_prediction(
            model_name=model_name,
            cwd=cwd,
            net_dimension=net_dimension,
            environment=env_instance,
        )
        
        print(f"Number of trades made: {num_trades}")
        episode_dates = np.unique(np.concatenate(episode_dates))
        episode_dates = np.concatenate(([episode_dates[0]], episode_dates))
        episode_dates = [timestamp.date() for timestamp in episode_dates]
        account_value_df = pd.DataFrame({'date': episode_dates, 'account_value': episode_total_assets})
        return account_value_df

## Import Stock Symbols and Indicators

In [ ]:
ticker_list = GP1_TICKER
action_dim = len(GP1_TICKER)

In [ ]:
print(ticker_list)

In [ ]:
print(INDICATORS)

In [ ]:
print(CDL)

## Calculate the DRL State Dimension Manually for Trading

In [ ]:
state_dim = 114

In [ ]:
state_dim

# Part 2: Train the Agent

Note: Start Date Must Be Monday for backtest plots

## Hyperparameter optimization Functions

In [ ]:
import optuna
import random
from datetime import datetime
from calendar import monthrange


env = StockTradingEnv

TRAIN_START_DATE = "2016-01-01"
TRAIN_END_DATE = "2016-12-31"

def optuna_train(
    trial,
    start_date,
    end_date,
    ticker_list,
    data_source,
    time_interval,
    technical_indicator_list,
    drl_lib,
    env,
    model_name,
    if_vix=True,
    if_cdl=True,
    **kwargs,
):
    # Create 'train_data' folder in the current working directory if it doesn't exist
    folder_path = os.path.join(os.getcwd(), "train_data")
    os.makedirs(folder_path, exist_ok=True)

    # Set the file paths within the 'train_data' folder
    data_file = os.path.join(folder_path, f"data_{start_date}_{end_date}.pkl")
    arrays_file = os.path.join(folder_path, f"arrays_{start_date}_{end_date}.pkl")

    if os.path.exists(data_file) and os.path.exists(arrays_file):
        # Load the saved data
        with open(data_file, "rb") as f:
            data = pickle.load(f)
        with open(arrays_file, "rb") as f:
            price_array, tech_array, turbulence_array, date_array = pickle.load(f)
    
    else:
        # download data
        dp = DataProcessor(data_source, **kwargs)
        data = dp.download_data(ticker_list, start_date, end_date, time_interval)
        data = dp.clean_data(data)
        data = dp.add_technical_indicator(data, technical_indicator_list)
        if if_cdl:
            data = dp.add_cdl(data)
        if if_vix:
            data = dp.add_vix(data)
        else:
            data = dp.add_turbulence(data)
        price_array, tech_array, turbulence_array, date_array = dp.df_to_array(data, if_vix, if_cdl)
        
        # Save the data and arrays
        with open(data_file, "wb") as f:
            pickle.dump(data, f)
        with open(arrays_file, "wb") as f:
            pickle.dump((price_array, tech_array, turbulence_array, date_array), f)
        
    env_config = {
        "price_array": price_array,
        "tech_array": tech_array,
        "turbulence_array": turbulence_array,
        "date_array": date_array,
        "if_train": True,
    }
    env_instance = env(config=env_config)

    # read parameters
    cwd = kwargs.get("cwd", "./" + str(model_name))
    
    if drl_lib == "elegantrl":
        DRLAgent_erl = DRLAgent
        break_step = kwargs.get("break_step", 1e6)
        erl_params = kwargs.get("erl_params")
        agent = DRLAgent_erl(
            env=env,
            price_array=price_array,
            tech_array=tech_array,
            turbulence_array=turbulence_array,
            date_array=date_array,
        )
        model = agent.get_model(model_name, model_kwargs=erl_params, cwd=cwd, model_resume_training=False)
        trained_model = agent.train_model(
            model=model, cwd=cwd, total_timesteps=break_step
        )

def optuna_test(
    trial,
    start_date,
    end_date,
    ticker_list,
    data_source,
    time_interval,
    technical_indicator_list,
    drl_lib,
    env,
    model_name,
    if_vix=True,
    if_cdl=True,
    **kwargs,
):
    # Create 'test_data' folder in the current working directory if it doesn't exist
    folder_path = os.path.join(os.getcwd(), "test_data")
    os.makedirs(folder_path, exist_ok=True)

    # Set the file paths within the 'test_data' folder
    data_file = os.path.join(folder_path, f"data_{start_date}_{end_date}.pkl")
    arrays_file = os.path.join(folder_path, f"arrays_{start_date}_{end_date}.pkl")

    if os.path.exists(data_file) and os.path.exists(arrays_file):
        # Load the saved data
        with open(data_file, "rb") as f:
            data = pickle.load(f)
        with open(arrays_file, "rb") as f:
            price_array, tech_array, turbulence_array, date_array = pickle.load(f)
    
    else:
        # download data
        dp = DataProcessor(data_source, **kwargs)
        data = dp.download_data(ticker_list, start_date, end_date, time_interval)
        data = dp.clean_data(data)
        data = dp.add_technical_indicator(data, technical_indicator_list)
        if if_cdl:
            data = dp.add_cdl(data)
        if if_vix:
            data = dp.add_vix(data)
        else:
            data = dp.add_turbulence(data)
        price_array, tech_array, turbulence_array, date_array = dp.df_to_array(data, if_vix, if_cdl)
        
        # Save the data and arrays
        with open(data_file, "wb") as f:
            pickle.dump(data, f)
        with open(arrays_file, "wb") as f:
            pickle.dump((price_array, tech_array, turbulence_array, date_array), f)
    env_config = {
        "price_array": price_array,
        "tech_array": tech_array,
        "turbulence_array": turbulence_array,
        "date_array": date_array,
        "if_train": False,
    }
    env_instance = env(config=env_config)

    # load elegantrl needs state dim, action dim and net dim
    net_dimension = kwargs.get("net_dimension")
    cwd = kwargs.get("cwd", "./" + str(model_name))
    print("price_array: ", len(price_array))

    if drl_lib == "elegantrl":
        DRLAgent_erl = DRLAgent
        episode_dates, episode_total_assets, num_trades = DRLAgent_erl.DRL_prediction(
            model_name=model_name,
            cwd=cwd,
            net_dimension=net_dimension,
            environment=env_instance,
        )
        
        print(f"Number of trades made: {num_trades}")
        episode_dates = np.unique(np.concatenate(episode_dates))
        episode_dates = np.concatenate(([episode_dates[0]], episode_dates))
        episode_dates = [timestamp.date() for timestamp in episode_dates]
        account_value_df = pd.DataFrame({'date': episode_dates, 'account_value': episode_total_assets})
        return account_value_df, num_trades


def optuna_calculate_loss(account_value_daily, w1=0.6, w2=0.4):
    pnl = account_value_daily['account_value'].iloc[-1] - account_value_daily['account_value'].iloc[0]
    neg_pnl = -pnl

    running_max = np.maximum.accumulate(account_value_daily['account_value'])
    drawdowns = (running_max - account_value_daily['account_value']) / running_max
    max_drawdown = np.max(drawdowns)

    loss = w1 * neg_pnl + w2 * max_drawdown
    return loss


def optuna_objective(trial):
    learning_rate = trial.suggest_float("learning_rate", 1e-6, 1e-5, log=True)
    weight_decay = trial.suggest_float("weight_decay", 1e-5, 0.15, log=True)
    lambda_entropy = trial.suggest_float("lambda_entropy", 1e-5, 0.20, log=True)
    batch_size = 4096
    gamma = 0.985
    net_dimension = [128, 1024]
    target_step = 5000
    eval_gap = 50
    eval_times = 1
    erl_params = {
        "learning_rate": learning_rate,
        "weight_decay": weight_decay,
        "lambda_entropy": lambda_entropy,
        "batch_size": batch_size,
        "gamma": gamma,
        "seed": 999,
        "net_dimension": net_dimension,
        "target_step": target_step,
        "eval_gap": eval_gap,
        "eval_times": eval_times,
    }
    
    random_year = random.randint(2017, 2022)
    random_month = random.randint(1, 12)
    _, last_day_of_month = monthrange(random_year, random_month)

    TEST_START_DATE = datetime(random_year, random_month, 1)
    TEST_END_DATE = datetime(random_year, random_month, last_day_of_month)
    TEST_START_DATE = TEST_START_DATE.strftime('%Y-%m-%d')
    TEST_END_DATE = TEST_END_DATE.strftime('%Y-%m-%d')
    
    optuna_train(
        trial,
        start_date = TRAIN_START_DATE, 
        end_date = TRAIN_END_DATE,
        ticker_list = ticker_list, 
        data_source = 'alpaca',
        time_interval = '1Min', 
        technical_indicator_list = INDICATORS,
        drl_lib ='elegantrl', 
        env = env,
        model_name ='ppo',
        if_vix = True,
        if_cdl = True, 
        API_KEY = ALPACA_API_KEY, 
        API_SECRET = ALPACA_API_SECRET, 
        API_BASE_URL = API_BASE_URL,
        cwd = './gp1_testing', #current_working_dir
        break_step = 2e5,
        erl_params = erl_params,
    )

    account_value_df, num_trades = optuna_test(
        trial,
        start_date = TEST_START_DATE, 
        end_date = TEST_END_DATE,
        ticker_list = ticker_list, 
        data_source = 'alpaca',
        time_interval = '1Min', 
        technical_indicator_list = INDICATORS,
        drl_lib = 'elegantrl', 
        env = env, 
        model_name = 'ppo',
        if_vix = True,
        if_cdl = True,
        API_KEY = ALPACA_API_KEY, 
        API_SECRET = ALPACA_API_SECRET, 
        API_BASE_URL = API_BASE_URL,
        cwd = './gp1_testing',
        net_dimension = erl_params['net_dimension'],
    )

    account_value_df['date'] = pd.to_datetime(account_value_df['date']).dt.date
    account_value_daily = account_value_df.groupby('date')['account_value'].last().reset_index()

    account_value_daily['daily_return'] = account_value_daily['account_value'].pct_change(1)

    if account_value_daily['daily_return'].std() > 1e-8:
        sharpe_ratio = (252**0.5) * account_value_daily['daily_return'].mean() / account_value_daily['daily_return'].std()
    else:
        sharpe_ratio = 0
    
    loss = optuna_calculate_loss(account_value_daily)

    return loss, sharpe_ratio, num_trades

## Run Optuna Trials

In [ ]:
study_name = "gp1_trials"
directions = ["minimize", "maximize", "minimize"]

study = optuna.create_study(study_name=study_name, directions=directions)

# timeout=3600 is 1 hr, 86400 is 24hrs or n_trials=100
study.optimize(optuna_objective, n_trials=5)

In [ ]:
# Get Pareto front trials
pareto_front_trials = study.best_trials

# Print Pareto front trials
print(f"Number of trials on the Pareto front: {len(study.best_trials)}")
for trial in pareto_front_trials:
    print(f"  Trial {trial.number}:")
    print("    Value: ", trial.values)
    print("    Params: ")
    for key, value in trial.params.items():
        print(f"      {key}: {value}")

In [ ]:
optuna.visualization.plot_pareto_front(study, target_names=["loss", "sharpe_ratio", "num_trades"])

In [ ]:
# Export Visualization
import plotly.io as pio


pareto_front = optuna.visualization.plot_pareto_front(study, target_names=["loss", "sharpe_ratio", "num_trades"])

pio.write_html(pareto_front, file='pareto_front.html', auto_open=False)


## Filter Trials

In [ ]:
trials = study.get_trials()

def trial_filter(trial):
    if trial.values is None:
        return False
    
    # order of objectives:
    # 0: loss (minimize)
    # 1: sharpe ratio (maximize)
    # 2: number of trades (minimize)
    
    # Set thresholds for each objective
    loss_threshold = 0
    sharpe_ratio_threshold = 3.0
    num_trades_threshold = 100

    if (trial.values[0] <= loss_threshold and 
        trial.values[1] >= sharpe_ratio_threshold and 
        trial.values[2] >= num_trades_threshold):
        return True
    return False

filtered_trials = [t for t in trials if trial_filter(t)]

## Filtered Trials Stats

In [ ]:
# Initialize variables for min, max, and sum
min_learning_rate = float('inf')
max_learning_rate = float('-inf')
sum_learning_rate = 0

min_weight_decay = float('inf')
max_weight_decay = float('-inf')
sum_weight_decay = 0

min_lambda_entropy = float('inf')
max_lambda_entropy = float('-inf')
sum_lambda_entropy = 0

# Iterate through the filtered trials and calculate the required statistics
num_trials = len(filtered_trials)

for trial in filtered_trials:
    learning_rate = trial.params['learning_rate']
    weight_decay = trial.params['weight_decay']
    lambda_entropy = trial.params['lambda_entropy']

    min_learning_rate = min(min_learning_rate, learning_rate)
    max_learning_rate = max(max_learning_rate, learning_rate)
    sum_learning_rate += learning_rate

    min_weight_decay = min(min_weight_decay, weight_decay)
    max_weight_decay = max(max_weight_decay, weight_decay)
    sum_weight_decay += weight_decay

    min_lambda_entropy = min(min_lambda_entropy, lambda_entropy)
    max_lambda_entropy = max(max_lambda_entropy, lambda_entropy)
    sum_lambda_entropy += lambda_entropy

# Calculate averages
avg_learning_rate = sum_learning_rate / num_trials
avg_weight_decay = sum_weight_decay / num_trials
avg_lambda_entropy = sum_lambda_entropy / num_trials

# Print the results
print("Learning rate: min={}, max={}, avg={}".format(min_learning_rate, max_learning_rate, avg_learning_rate))
print("Weight decay: min={}, max={}, avg={}".format(min_weight_decay, max_weight_decay, avg_weight_decay))
print("Lambda entropy: min={}, max={}, avg={}".format(min_lambda_entropy, max_lambda_entropy, avg_lambda_entropy))

## The Gauntlet

In [ ]:
import os
import pickle
import shutil
import numpy as np
import pandas as pd
from datetime import datetime
from dateutil.rrule import rrule, MONTHLY
from dateutil.parser import parse
from calendar import monthrange

import optuna


env = StockTradingEnv

TRAIN_START_DATE = "2016-01-01"
TRAIN_END_DATE = "2016-12-31"
TEST_START_DATE = "2017-01-01"
TEST_END_DATE = "2017-01-31"
END_DATE = '2023-03-31'

def optuna_train(
    trial,
    start_date,
    end_date,
    ticker_list,
    data_source,
    time_interval,
    technical_indicator_list,
    drl_lib,
    env,
    model_name,
    if_vix=True,
    if_cdl=True,
    **kwargs,
):
    # Create 'train_data' folder in the current working directory if it doesn't exist
    folder_path = os.path.join(os.getcwd(), "train_data")
    os.makedirs(folder_path, exist_ok=True)

    # Set the file paths within the 'train_data' folder
    data_file = os.path.join(folder_path, f"data_{start_date}_{end_date}.pkl")
    arrays_file = os.path.join(folder_path, f"arrays_{start_date}_{end_date}.pkl")

    if os.path.exists(data_file) and os.path.exists(arrays_file):
        # Load the saved data
        with open(data_file, "rb") as f:
            data = pickle.load(f)
        with open(arrays_file, "rb") as f:
            price_array, tech_array, turbulence_array, date_array = pickle.load(f)
    
    else:
        # download data
        dp = DataProcessor(data_source, **kwargs)
        data = dp.download_data(ticker_list, start_date, end_date, time_interval)
        data = dp.clean_data(data)
        data = dp.add_technical_indicator(data, technical_indicator_list)
        if if_cdl:
            data = dp.add_cdl(data)
        if if_vix:
            data = dp.add_vix(data)
        else:
            data = dp.add_turbulence(data)
        price_array, tech_array, turbulence_array, date_array = dp.df_to_array(data, if_vix, if_cdl)
        
        # Save the data and arrays
        with open(data_file, "wb") as f:
            pickle.dump(data, f)
        with open(arrays_file, "wb") as f:
            pickle.dump((price_array, tech_array, turbulence_array, date_array), f)
        
    env_config = {
        "price_array": price_array,
        "tech_array": tech_array,
        "turbulence_array": turbulence_array,
        "date_array": date_array,
        "if_train": True,
    }
    env_instance = env(config=env_config)

    # read parameters
    cwd = kwargs.get("cwd", "./" + str(model_name))

    if drl_lib == "elegantrl":
        DRLAgent_erl = DRLAgent
        break_step = kwargs.get("break_step", 1e6)
        erl_params = kwargs.get("erl_params")
        agent = DRLAgent_erl(
            env=env,
            price_array=price_array,
            tech_array=tech_array,
            turbulence_array=turbulence_array,
            date_array=date_array,
        )
        model = agent.get_model(model_name, model_kwargs=erl_params, cwd=cwd, model_resume_training=True)
        trained_model = agent.train_model(
            model=model, cwd=cwd, total_timesteps=break_step
        )

def optuna_test(
    trial,
    start_date,
    end_date,
    ticker_list,
    data_source,
    time_interval,
    technical_indicator_list,
    drl_lib,
    env,
    model_name,
    if_vix=True,
    if_cdl=True,
    **kwargs,
):
    # Create 'test_data' folder in the current working directory if it doesn't exist
    folder_path = os.path.join(os.getcwd(), "test_data")
    os.makedirs(folder_path, exist_ok=True)

    # Set the file paths within the 'test_data' folder
    data_file = os.path.join(folder_path, f"data_{start_date}_{end_date}.pkl")
    arrays_file = os.path.join(folder_path, f"arrays_{start_date}_{end_date}.pkl")

    if os.path.exists(data_file) and os.path.exists(arrays_file):
        # Load the saved data
        with open(data_file, "rb") as f:
            data = pickle.load(f)
        with open(arrays_file, "rb") as f:
            price_array, tech_array, turbulence_array, date_array = pickle.load(f)
    
    else:
        # download data
        dp = DataProcessor(data_source, **kwargs)
        data = dp.download_data(ticker_list, start_date, end_date, time_interval)
        data = dp.clean_data(data)
        data = dp.add_technical_indicator(data, technical_indicator_list)
        if if_cdl:
            data = dp.add_cdl(data)
        if if_vix:
            data = dp.add_vix(data)
        else:
            data = dp.add_turbulence(data)
        price_array, tech_array, turbulence_array, date_array = dp.df_to_array(data, if_vix, if_cdl)
        
        # Save the data and arrays
        with open(data_file, "wb") as f:
            pickle.dump(data, f)
        with open(arrays_file, "wb") as f:
            pickle.dump((price_array, tech_array, turbulence_array, date_array), f)
            
    env_config = {
        "price_array": price_array,
        "tech_array": tech_array,
        "turbulence_array": turbulence_array,
        "date_array": date_array,
        "if_train": False,
    }
    env_instance = env(config=env_config)

    # load elegantrl needs state dim, action dim and net dim
    net_dimension = kwargs.get("net_dimension")
    cwd = kwargs.get("cwd", "./" + str(model_name))
    print("price_array: ", len(price_array))

    if drl_lib == "elegantrl":
        DRLAgent_erl = DRLAgent
        episode_dates, episode_total_assets, num_trades = DRLAgent_erl.DRL_prediction(
            model_name=model_name,
            cwd=cwd,
            net_dimension=net_dimension,
            environment=env_instance,
        )
        
        print(f"Number of trades made: {num_trades}")
        episode_dates = np.unique(np.concatenate(episode_dates))
        episode_dates = np.concatenate(([episode_dates[0]], episode_dates))
        episode_dates = [timestamp.date() for timestamp in episode_dates]
        account_value_df = pd.DataFrame({'date': episode_dates, 'account_value': episode_total_assets})
        return account_value_df, num_trades


def optuna_calculate_loss(account_value_daily, w1=0.6, w2=0.4):
    pnl = account_value_daily['account_value'].iloc[-1] - account_value_daily['account_value'].iloc[0]
    neg_pnl = -pnl

    running_max = np.maximum.accumulate(account_value_daily['account_value'])
    drawdowns = (running_max - account_value_daily['account_value']) / running_max
    max_drawdown = np.max(drawdowns)

    loss = w1 * neg_pnl + w2 * max_drawdown
    return loss

def optuna_gauntlet_objective(trial):
    loss_threshold = 0
    sharpe_threshold = 0
    min_trades_threshold = 100
    max_trades_threshold = 5_000
    returns_threshold = 1.0
    monthly_returns = []
    monthly_loss = []
    monthly_sharpe = []
    monthly_trades = []
    
    
    train_start_date = datetime.strptime(TRAIN_START_DATE, '%Y-%m-%d').date()
    train_end_date = datetime.strptime(TRAIN_END_DATE, '%Y-%m-%d').date()
    test_start_date = datetime.strptime(TEST_START_DATE, '%Y-%m-%d').date()
    test_end_date = datetime.strptime(TEST_END_DATE, '%Y-%m-%d').date()
    end_date = datetime.strptime(END_DATE, '%Y-%m-%d').date()
    
    while train_end_date <= end_date:
        learning_rate = trial.suggest_float("learning_rate", 1e-6, 1e-5, log=True)
        weight_decay = trial.suggest_float("weight_decay", 1e-5, 0.15, log=True)
        lambda_entropy = trial.suggest_float("lambda_entropy", 1e-5, 0.20, log=True)
        batch_size = 4096
        gamma = 0.985
        net_dimension = [128, 1024]
        target_step = 5000
        eval_gap = 50
        eval_times = 1
        erl_params = {
            "learning_rate": learning_rate,
            "weight_decay": weight_decay,
            "lambda_entropy": lambda_entropy,
            "batch_size": batch_size,
            "gamma": gamma,
            "seed": 999,
            "net_dimension": net_dimension,
            "target_step": target_step,
            "eval_gap": eval_gap,
            "eval_times": eval_times,
        }
        
        optuna_train(
            trial,
            start_date=train_start_date.strftime('%Y-%m-%d'),
            end_date=train_end_date.strftime('%Y-%m-%d'),
            ticker_list = ticker_list, 
            data_source = 'alpaca',
            time_interval= '1Min', 
            technical_indicator_list= INDICATORS,
            drl_lib='elegantrl', 
            env=env,
            model_name='ppo',
            if_vix=True,
            if_cdl=True, 
            API_KEY = ALPACA_API_KEY, 
            API_SECRET = ALPACA_API_SECRET, 
            API_BASE_URL = API_BASE_URL,
            cwd='./gp1_testing', #current_working_dir
            break_step=2e5,
            erl_params=erl_params
        )

        account_value_df, num_trades = optuna_test(
            trial,
            start_date=test_start_date.strftime('%Y-%m-%d'),
            end_date=test_end_date.strftime('%Y-%m-%d'),
            ticker_list = ticker_list, 
            data_source = 'alpaca',
            time_interval= '1Min', 
            technical_indicator_list= INDICATORS,
            drl_lib='elegantrl', 
            env=env, 
            model_name='ppo',
            if_vix=True,
            if_cdl=True,
            API_KEY = ALPACA_API_KEY, 
            API_SECRET = ALPACA_API_SECRET, 
            API_BASE_URL = API_BASE_URL,
            cwd='./gp1_testing',
            net_dimension = erl_params['net_dimension'],
        )
        # Stats
        account_value_df['date'] = pd.to_datetime(account_value_df['date']).dt.date
        account_value_daily = account_value_df.groupby('date')['account_value'].last().reset_index()

        account_value_daily['daily_return'] = account_value_daily['account_value'].pct_change(1)

        if account_value_daily['daily_return'].std() > 1e-8:
            sharpe_ratio = (252**0.5) * account_value_daily['daily_return'].mean() / account_value_daily['daily_return'].std()
        else:
            sharpe_ratio = 0
        
        loss = optuna_calculate_loss(account_value_daily)

        initial_account_value = account_value_daily['account_value'].iloc[0]
        final_account_value = account_value_daily['account_value'].iloc[-1]

        monthly_return = (final_account_value - initial_account_value) / initial_account_value
        returns = monthly_return * 100
        
        # Check if all thresholds are met
        if (
            loss < loss_threshold
            and sharpe_ratio > sharpe_threshold
            and min_trades_threshold <= num_trades <= max_trades_threshold
            and returns >= returns_threshold
        ):
            # Record Stats
            monthly_returns.append(returns)
            monthly_loss.append(loss)
            monthly_sharpe.append(sharpe_ratio)
            monthly_trades.append(num_trades)
            
            # Move the date range one month forward
            train_start_date = test_start_date
            train_end_date = test_end_date
            test_start_date = list(rrule(freq=MONTHLY, dtstart=parse(test_start_date.strftime('%Y-%m-%d')), count=2))[-1].date()
            _, last_day_of_month = monthrange(test_start_date.year, test_start_date.month)
            test_end_date = test_start_date.replace(day=last_day_of_month)
        else:
            # Delete the model if saved to disk
            if os.path.exists("./gp1_testing/actor.pth"):
                os.remove("./gp1_testing/actor.pth")
            # Cancel Trial
            raise optuna.TrialPruned()
    avg_monthly_loss = np.mean(monthly_loss)
    avg_monthly_sharpe = np.mean(monthly_sharpe)
    avg_monthly_trades = np.mean(monthly_trades)
    avg_monthly_return = np.mean(monthly_returns)
    
    # Move and rename the model file
    src_file = "./gp1_testing/actor.pth"
    dst_dir = "./gp1_gauntlet"
    os.makedirs(dst_dir, exist_ok=True)
    dst_file = os.path.join(dst_dir, f"actor_trial_{trial.number}_avg_return_{int(round(avg_monthly_return))}.pth")
    shutil.move(src_file, dst_file)
    
    return avg_monthly_loss, avg_monthly_sharpe, avg_monthly_trades

In [ ]:
study_name = "gp1_gauntlet"
directions = ["minimize", "maximize", "minimize"]

study = optuna.create_study(study_name=study_name, directions=directions)

# timeout=3600 is 1 hr, 86400 is 24hrs or n_trials=100
study.optimize(optuna_gauntlet_objective, timeout=(3600 * 22))

In [ ]:
optuna.visualization.plot_pareto_front(study, target_names=["avg_monthly_loss", "avg_monthly_sharpe", "avg_monthly_trades"])

In [ ]:
# Export Visualization
import plotly.io as pio


pareto_front = optuna.visualization.plot_pareto_front(study, target_names=["avg_monthly_loss", "avg_monthly_sharpe", "avg_monthly_trades"])

pio.write_html(pareto_front, file='pareto_front.html', auto_open=False)

## Update Model

# Part 3: Backtest

In [ ]:
ERL_PARAMS = {
    "learning_rate": 2e-5,
    "batch_size": 2048,
    "gamma":  0.985,
    "seed":42,
    "net_dimension":[128, 256], 
    "target_step":5000, 
    "eval_gap":50,
    "eval_times":1
}
env = StockTradingEnv

TEST_START_DATE = "2023-01-02"
TEST_END_DATE = "2023-03-31"

gp1_account_value_df=test(start_date = TEST_START_DATE, 
                    end_date = TEST_END_DATE,
                    ticker_list = ticker_list, 
                    data_source = 'alpaca',
                    time_interval= '1Min', 
                    technical_indicator_list= INDICATORS,
                    drl_lib='elegantrl', 
                    env=env, 
                    model_name='ppo',
                    if_vix=True,
                    if_cdl=True,
                    API_KEY = ALPACA_API_KEY, 
                    API_SECRET = ALPACA_API_SECRET, 
                    API_BASE_URL = API_BASE_URL,
                    cwd='./gp1_trained',
                    net_dimension = ERL_PARAMS['net_dimension'])

In [ ]:
print("==============Get Baseline Stats===========")

# S&P 500: ^GSPC
# Dow Jones Index: ^DJI
# NASDAQ 100: ^NDX

baseline_df = get_baseline(
        ticker="^GSPC", 
        start = TEST_START_DATE,
        end = TEST_END_DATE)

stats = backtest_stats(baseline_df, value_col_name = 'close')

In [ ]:
print("==============GP1 Stats===========")

stats = backtest_stats(gp1_account_value_df)

## Charts

In [ ]:
%matplotlib inline

gp1_account_value_df = drop_dup_dates(gp1_account_value_df)

# S&P 500: ^GSPC
# Dow Jones Index: ^DJI
# NASDAQ 100: ^NDX

backtest_plot(gp1_account_value_df, 
             baseline_ticker = '^GSPC', 
             baseline_start = TEST_START_DATE,
             baseline_end = TEST_END_DATE,)

# Part 4: Deploy the Agent

## Setup Alpaca Trading Environment

In [ ]:
import datetime
import threading
from processor_alpaca import AlpacaProcessor
import alpaca_trade_api as tradeapi
import time
import pandas as pd
import numpy as np
import torch
import gymnasium as gym

class AlpacaPaperTrading():

    def __init__(self,ticker_list, time_interval, drl_lib, agent, cwd, net_dim, 
                 state_dim, action_dim, API_KEY, API_SECRET, 
                 API_BASE_URL, tech_indicator_list, turbulence_thresh=99, 
                 max_stock=1e2, latency = None):
        #load agent
        self.drl_lib = drl_lib
        if agent =='ppo':
            if drl_lib == 'elegantrl':              
                agent_class = AgentPPO
                agent = agent_class(net_dim, state_dim, action_dim)
                actor = agent.act
                # load agent
                try:  
                    cwd = cwd + '/actor.pth'
                    print(f"| load actor from: {cwd}")
                    actor.load_state_dict(torch.load(cwd, map_location=lambda storage, loc: storage))
                    self.act = actor
                    self.device = agent.device
                except BaseException:
                    raise ValueError("Fail to load agent!")
                    
            else:
                raise ValueError('The DRL library input is NOT supported yet. Please check your input.')
               
        else:
            raise ValueError('Agent input is NOT supported yet.')
            
            
            
        #connect to Alpaca trading API
        try:
            self.alpaca = tradeapi.REST(API_KEY,API_SECRET,API_BASE_URL, 'v2')
        except:
            raise ValueError('Fail to connect Alpaca. Please check account info and internet connection.')
        
        #read trading time interval
        if time_interval == '1s':
            self.time_interval = 1
        elif time_interval == '5s':
            self.time_interval = 5
        elif time_interval == '1Min':
            self.time_interval = 60
        elif time_interval == '5Min':
            self.time_interval = 60 * 5
        elif time_interval == '15Min':
            self.time_interval = 60 * 15
        else:
            raise ValueError('Time interval input is NOT supported yet.')
        
        #read trading settings
        self.tech_indicator_list = tech_indicator_list
        self.turbulence_thresh = turbulence_thresh
        self.max_stock = max_stock 
        
        #initialize account
        self.stocks = np.asarray([0] * len(ticker_list)) #stocks holding
        self.stocks_cd = np.zeros_like(self.stocks) 
        self.cash = None #cash record 
        self.stocks_df = pd.DataFrame(self.stocks, columns=['stocks'], index = ticker_list)
        self.assets_list = []
        self.price = np.asarray([0] * len(ticker_list))
        self.stockUniverse = ticker_list
        self.turbulence_bool = 0
        self.equities = []
        
    def test_latency(self, test_times = 10): 
        total_time = 0
        for i in range(0, test_times):
            time0 = time.time()
            self.get_state()
            time1 = time.time()
            temp_time = time1 - time0
            total_time += temp_time
        latency = total_time/test_times
        print('latency for data processing: ', latency)
        return latency
        
    def run(self):
        orders = self.alpaca.list_orders(status="open")
        for order in orders:
          self.alpaca.cancel_order(order.id)
    
        # Wait for market to open.
        print("Waiting for market to open...")
        self.awaitMarketOpen()
        print("Market opened.")

        while True:

          # Figure out when the market will close so we can prepare to sell beforehand.
          clock = self.alpaca.get_clock()
          closingTime = clock.next_close.replace(tzinfo=datetime.timezone.utc).timestamp()
          currTime = clock.timestamp.replace(tzinfo=datetime.timezone.utc).timestamp()
          self.timeToClose = closingTime - currTime
    
          if(self.timeToClose < (60)):
            # Close all positions when 1 minutes til market close.
            print("Market closing soon. Stop trading.")
            break
            
            '''# Close all positions when 1 minutes til market close.
            print("Market closing soon.  Closing positions.")

            threads = []
            positions = self.alpaca.list_positions()
            for position in positions:
              if(position.side == 'long'):
                orderSide = 'sell'
              else:
                orderSide = 'buy'
              qty = abs(int(float(position.qty)))
              respSO = []
              tSubmitOrder = threading.Thread(target=self.submitOrder(qty, position.symbol, orderSide, respSO))
              tSubmitOrder.start()
              threads.append(tSubmitOrder)    # record thread for joining later

            for x in threads:   #  wait for all threads to complete
                x.join()     
            # Run script again after market close for next trading day.
            print("Sleeping until market close (15 minutes).")
            time.sleep(60 * 15)'''
            
          else:
            self.trade()
            last_equity = float(self.alpaca.get_account().last_equity)
            cur_time = time.time()
            self.equities.append([cur_time,last_equity])
            time.sleep(self.time_interval)
            
    def awaitMarketOpen(self):
        isOpen = self.alpaca.get_clock().is_open
        while(not isOpen):
          clock = self.alpaca.get_clock()
          openingTime = clock.next_open.replace(tzinfo=datetime.timezone.utc).timestamp()
          currTime = clock.timestamp.replace(tzinfo=datetime.timezone.utc).timestamp()
          timeToOpen = int((openingTime - currTime) / 60)
          print(str(timeToOpen) + " minutes til market open.")
          time.sleep(60)
          isOpen = self.alpaca.get_clock().is_open
    
    def trade(self):
        state = self.get_state()
        
        if self.drl_lib == 'elegantrl':
            with torch.no_grad():
                s_tensor = torch.as_tensor((state,), device=self.device)
                a_tensor = self.act(s_tensor)  
                action = a_tensor.detach().cpu().numpy()[0]  
            action = (action * self.max_stock).astype(int)
            
        elif self.drl_lib == 'rllib':
            action = self.agent.compute_single_action(state)
        
        elif self.drl_lib == 'stable_baselines3':
            action = self.model.predict(state)[0]
            
        else:
            raise ValueError('The DRL library input is NOT supported yet. Please check your input.')
        
        self.stocks_cd += 1
        if self.turbulence_bool == 0:
            min_action = 10  # stock_cd
            threads = []
            for index in np.where(action < -min_action)[0]:  # sell_index:
                sell_num_shares = min(self.stocks[index], -action[index])
                qty =  abs(int(sell_num_shares))
                respSO = []
                tSubmitOrder = threading.Thread(target=self.submitOrder(qty, self.stockUniverse[index], 'sell', respSO))
                tSubmitOrder.start()
                threads.append(tSubmitOrder)    # record thread for joining later
                self.cash = float(self.alpaca.get_account().cash)
                self.stocks_cd[index] = 0
            
            for x in threads:   #  wait for all threads to complete
                x.join()     

            threads = []
            for index in np.where(action > min_action)[0]:  # buy_index:
                if self.cash < 0:
                    tmp_cash = 0
                else:
                    tmp_cash = self.cash
                buy_num_shares = min(tmp_cash // self.price[index], abs(int(action[index])))
                if (buy_num_shares != buy_num_shares): # if buy_num_change = nan
                    qty = 0 # set to 0 quantity
                else:
                    qty = abs(int(buy_num_shares))
                qty = abs(int(buy_num_shares))
                respSO = []
                tSubmitOrder = threading.Thread(target=self.submitOrder(qty, self.stockUniverse[index], 'buy', respSO))
                tSubmitOrder.start()
                threads.append(tSubmitOrder)    # record thread for joining later
                self.cash = float(self.alpaca.get_account().cash)
                self.stocks_cd[index] = 0

            for x in threads:   #  wait for all threads to complete
                x.join()     
                
        else:  # sell all when turbulence
            threads = []
            positions = self.alpaca.list_positions()
            for position in positions:
                if(position.side == 'long'):
                    orderSide = 'sell'
                else:
                    orderSide = 'buy'
                qty = abs(int(float(position.qty)))
                respSO = []
                tSubmitOrder = threading.Thread(target=self.submitOrder(qty, position.symbol, orderSide, respSO))
                tSubmitOrder.start()
                threads.append(tSubmitOrder)    # record thread for joining later

            for x in threads:   #  wait for all threads to complete
                x.join()     
            
            self.stocks_cd[:] = 0
            
    
    def get_state(self):
        alpaca = AlpacaProcessor(api=self.alpaca)
        price, tech, turbulence = alpaca.fetch_latest_data(ticker_list = self.stockUniverse, time_interval='1Min',
                                                     tech_indicator_list=self.tech_indicator_list)
        turbulence_bool = 1 if turbulence >= self.turbulence_thresh else 0
        
        turbulence = (self.sigmoid_sign(turbulence, self.turbulence_thresh) * 2 ** -5).astype(np.float32)
        
        tech = tech * 2 ** -7
        positions = self.alpaca.list_positions()
        stocks = [0] * len(self.stockUniverse)
        for position in positions:
            ind = self.stockUniverse.index(position.symbol)
            stocks[ind] = ( abs(int(float(position.qty))))
        
        stocks = np.asarray(stocks, dtype = float)
        cash = float(self.alpaca.get_account().cash)
        self.cash = cash
        self.stocks = stocks
        self.turbulence_bool = turbulence_bool 
        self.price = price
        
        
        
        amount = np.array(self.cash * (2 ** -12), dtype=np.float32)
        scale = np.array(2 ** -6, dtype=np.float32)
        state = np.hstack((amount,
                    turbulence,
                    self.turbulence_bool,
                    price * scale,
                    self.stocks * scale,
                    self.stocks_cd,
                    tech,
                    )).astype(np.float32)
        state[np.isnan(state)] = 0.0
        state[np.isinf(state)] = 0.0
        print(len(self.stockUniverse))
        return state
        
    def submitOrder(self, qty, stock, side, resp):
        if(qty > 0):
          try:
            self.alpaca.submit_order(stock, qty, side, "market", "day")
            print("Market order of | " + str(qty) + " " + stock + " " + side + " | completed.")
            resp.append(True)
          except:
            print("Order of | " + str(qty) + " " + stock + " " + side + " | did not go through.")
            resp.append(False)
        else:
          print("Quantity is 0, order of | " + str(qty) + " " + stock + " " + side + " | not completed.")
          resp.append(True)

    @staticmethod
    def sigmoid_sign(ary, thresh):
        def sigmoid(x):
            return 1 / (1 + np.exp(-x * np.e)) - 0.5

        return sigmoid(ary / thresh) * thresh

## Run Trading

In [ ]:
print(GP1_TICKER)

In [ ]:
state_dim

In [ ]:
action_dim

In [ ]:
ERL_PARAMS = {
    "learning_rate": 2e-5,
    "batch_size": 2048,
    "gamma":  0.985,
    "seed":42,
    "net_dimension":[128, 256], 
    "target_step":5000, 
    "eval_gap":50,
    "eval_times":1
}

In [ ]:
trading_erl = AlpacaPaperTrading(ticker_list = GP1_TICKER, 
                                       time_interval = '1Min', 
                                       drl_lib = 'elegantrl', 
                                       agent = 'ppo', 
                                       cwd = './gp1_trained', 
                                       net_dim = ERL_PARAMS['net_dimension'], 
                                       state_dim = state_dim, 
                                       action_dim= action_dim, 
                                       API_KEY = ALPACA_API_KEY, 
                                       API_SECRET = ALPACA_API_SECRET, 
                                       API_BASE_URL = API_BASE_URL, 
                                       tech_indicator_list = INDICATORS,
                                       max_stock=1e2, turbulence_thresh=99)
trading_erl.run()

# Part 5: Check Portfolio Performance

In [ ]:
import alpaca_trade_api as tradeapi
import exchange_calendars as tc
import numpy as np
import pandas as pd
import pytz
import yfinance as yf
import matplotlib.ticker as ticker
import matplotlib.dates as mdates
from datetime import datetime as dt
from plot import backtest_stats
import matplotlib.pyplot as plt

In [ ]:
def get_trading_days(start, end):
    nyse = tc.get_calendar('NYSE')
    df = nyse.sessions_in_range(pd.Timestamp(start,tz=pytz.UTC),
                                pd.Timestamp(end,tz=pytz.UTC))
    trading_days = []
    for day in df:
        trading_days.append(str(day)[:10])

    return trading_days

def alpaca_history(key, secret, url, start, end):
    api = tradeapi.REST(key, secret, url, 'v2')
    trading_days = get_trading_days(start, end)
    df = pd.DataFrame()
    for day in trading_days:
        df = df.append(api.get_portfolio_history(date_start = day,timeframe='5Min').df.iloc[:78])
    equities = df.equity.values
    cumu_returns = equities/equities[0]
    cumu_returns = cumu_returns[~np.isnan(cumu_returns)]
    
    return df, cumu_returns

def SP500_history(start):
    data_df = yf.download(['^GSPC'],start=start, interval="5m")
    data_df = data_df.iloc[:]
    baseline_returns = data_df['Adj Close'].values/data_df['Adj Close'].values[0]
    return data_df, baseline_returns

## Get cumulative return

In [ ]:
df_erl, cumu_erl = alpaca_history(key=ALPACA_API_KEY, 
                                  secret=ALPACA_API_SECRET, 
                                  url=API_BASE_URL, 
                                  start='2023-03-24', #must be within 1 month
                                  end='2023-03-24') #change the date if error occurs

In [ ]:
df_sp500, cumu_sp500 = SP500_history(start='2023-01-01')

In [ ]:
df_erl.tail()

In [ ]:
returns_erl = cumu_erl -1 
returns_sp500 = cumu_sp500 - 1
returns_sp500 = returns_sp500[:returns_erl.shape[0]]
print('len of erl return: ', returns_erl.shape[0])
print('len of sp500 return: ', returns_sp500.shape[0])

In [ ]:
returns_erl

## plot and save

In [ ]:
import matplotlib.pyplot as plt
plt.figure(dpi=1000)
plt.grid()
plt.grid(which='minor', axis='y')
plt.title('Stock Trading (Paper trading)', fontsize=20)
plt.plot(returns_erl, label = 'ElegantRL Agent', color = 'red')
#plt.plot(returns_sb3, label = 'Stable-Baselines3 Agent', color = 'blue' )
#plt.plot(returns_rllib, label = 'RLlib Agent', color = 'green')
plt.plot(returns_sp500, label = 'S&P 500', color = 'grey')
plt.ylabel('Return', fontsize=16)
plt.xlabel('Year 2023', fontsize=16)
plt.xticks(size = 14)
plt.yticks(size = 14)
ax = plt.gca()
ax.xaxis.set_major_locator(ticker.MultipleLocator(78))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(6))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(0.005))
ax.yaxis.set_major_formatter(ticker.PercentFormatter(xmax=1, decimals=2))
ax.xaxis.set_major_formatter(ticker.FixedFormatter(['','10-19','','10-20',
                                                    '','10-21','','10-22']))
plt.legend(fontsize=10.5)
plt.savefig('papertrading_stock.png')